In [1]:
import librosa
import numpy as np

def extract_features(file_path):
    """Extract audio features for TensorFlow model input."""
    y, sr = librosa.load(file_path, duration=30)  # Load first 30 seconds for consistency
    
    # Extract audio features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=y)
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    
    # Aggregate features (mean and std for each)
    features = np.hstack([
        np.mean(mfcc, axis=1), np.std(mfcc, axis=1),
        np.mean(spectral_centroid), np.std(spectral_centroid),
        np.mean(spectral_rolloff), np.std(spectral_rolloff),
        np.mean(zero_crossing_rate), np.std(zero_crossing_rate),
        tempo
    ])
    return features

In [2]:
import pandas as pd
import os

def prepare_dataset(csv_path, audio_folder):
    """Prepare features and labels for model training."""
    data = pd.read_csv(csv_path)  # CSV with columns: 'file', 'label'
    X, y = [], []
    for _, row in data.iterrows():
        file_path = os.path.join(audio_folder, row['file'])
        features = extract_features(file_path)
        X.append(features)
        y.append(row['label'])
    return np.array(X), np.array(y)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def build_model(input_shape, num_classes):
    """Build a simple TensorFlow model."""
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')  # For multi-class classification
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [4]:
# Load dataset
csv_path = "dataset.csv"  # CSV file with 'file' and 'label' columns
audio_folder = "audio_files"
X, y = prepare_dataset(csv_path, audio_folder)

# Build and train the model
model = build_model(X.shape[1], num_classes=len(np.unique(y)))
history = model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

# Save the model
model.save("sentiment_model.h5")


FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'

In [ ]:
def predict_sentiment(model_path, audio_path):
    """Predict the sentiment of a song."""
    # Load the model
    model = tf.keras.models.load_model(model_path)
    
    # Extract features from audio
    features = extract_features(audio_path).reshape(1, -1)
    
    # Predict sentiment
    prediction = model.predict(features)
    sentiment = np.argmax(prediction)
    print(f"Predicted sentiment: {sentiment}")
    return sentiment

# Example usage
audio_path = "test_song.mp3"
predict_sentiment("sentiment_model.h5", audio_path)
